## Embedding a Mayavi plot

- Similar approach to chaco
- Mayavi provides:
  - `MlabSceneModel`: Like Chaco's Plot.
  - `SceneEditor`: Like the ComponentEditor
- Examples


In [ ]:
# %load code/mayavi_simple.py
import numpy as np
from traits.api import HasTraits, Instance, observe
from traitsui.api import Item, View
from mayavi.core.ui.api import MayaviScene, MlabSceneModel, SceneEditor, DecoratedScene


class Plot3D(HasTraits):
    scene = Instance(MlabSceneModel, args=())
    view = View(Item(name='scene',
                     editor=SceneEditor(scene_class=DecoratedScene),
                     show_label=False, resizable=True,
                     height=500, width=500),
                resizable=True)

    @observe('scene.activated')
    def generate_data(self, event=None):
        # Create some data
        x, y = np.mgrid[-2:2:100j, -2:2:100j]
        r = 10*np.sqrt(x**2 + y**2)
        z = np.sin(r)/r
        self.scene.mlab.surf(x, y, z, colormap='gist_earth')


if __name__ == '__main__':
    Plot3D().configure_traits()


## Another simple example


In [ ]:
# %load code/mlab_interactive_dialog.py
from numpy import arange, pi, cos, sin

from traits.api import HasTraits, Range, Instance, observe
from traitsui.api import View, Item, Group

from mayavi.core.api import PipelineBase
from mayavi.core.ui.api import MayaviScene, SceneEditor, \
                MlabSceneModel


dphi = pi/1000.
phi = arange(0.0, 2*pi + 0.5*dphi, dphi, 'd')


def curve(n_mer, n_long):
    mu = phi*n_mer
    x = cos(mu) * (1 + cos(n_long * mu/n_mer)*0.5)
    y = sin(mu) * (1 + cos(n_long * mu/n_mer)*0.5)
    z = 0.5 * sin(n_long*mu/n_mer)
    t = sin(mu)
    return x, y, z, t


class MyModel(HasTraits):
    n_meridional = Range(0, 30, 6)
    n_longitudinal = Range(0, 30, 11)

    scene = Instance(MlabSceneModel, ())

    plot = Instance(PipelineBase)


    # When the scene is activated, or when the parameters are changed, we
    # update the plot.
    @observe('n_meridional,n_longitudinal,scene.activated')
    def update_plot(self, event=None):
        x, y, z, t = curve(self.n_meridional, self.n_longitudinal)
        if self.plot is None:
            self.plot = self.scene.mlab.plot3d(
                x, y, z, t, tube_radius=0.025, colormap='Spectral'
            )
        else:
            self.plot.mlab_source.trait_set(x=x, y=y, z=z, scalars=t)

    # The layout of the dialog created
    view = View(Item('scene', editor=SceneEditor(scene_class=MayaviScene),
                     height=250, width=300, show_label=False),
                Group(
                        '_', 'n_meridional', 'n_longitudinal',
                     ),
                resizable=True,
                )


my_model = MyModel()
my_model.configure_traits()


## More complex example


In [ ]:
%load code/lorenz_ui.py

## Exercise

- Pick some problem we solved
- Plot it using Mayavi with a useful UI
